## Importing Modules

In [181]:
import yfinance as yf
import pandas as pd
import numpy as np
import tensorflow as tf
import keras_tuner as kt
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
tf.device("/GPU:0")


## Training The Dataset and Normalizing It

In [182]:
# Fetch AAPL stock data with a 1-hour timeframe
nsei = yf.Ticker("^NSEI")
dataset_train = nsei.history(period="max", interval="1d")
dataset_train

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2007-09-17 00:00:00+05:30,4518.450195,4549.049805,4482.850098,4494.649902,0,0.0,0.0
2007-09-18 00:00:00+05:30,4494.100098,4551.799805,4481.549805,4546.200195,0,0.0,0.0
2007-09-19 00:00:00+05:30,4550.250000,4739.000000,4550.250000,4732.350098,0,0.0,0.0
2007-09-20 00:00:00+05:30,4734.850098,4760.850098,4721.149902,4747.549805,0,0.0,0.0
2007-09-21 00:00:00+05:30,4752.950195,4855.700195,4733.700195,4837.549805,0,0.0,0.0
...,...,...,...,...,...,...,...
2024-11-08 00:00:00+05:30,24207.699219,24276.150391,24066.650391,24148.199219,298700,0.0,0.0
2024-11-11 00:00:00+05:30,24087.250000,24336.800781,24004.599609,24141.300781,273400,0.0,0.0
2024-11-12 00:00:00+05:30,24225.800781,24242.000000,23839.150391,23883.449219,255800,0.0,0.0


In [183]:
training_set = dataset_train.iloc[:-33, 1:2].values
training_set.shape

(4175, 1)

In [184]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(-1,1))
training_set_scaled = sc.fit_transform(training_set)
training_set_scaled

array([[-0.83422711],
       [-0.83399497],
       [-0.81819218],
       ...,
       [ 0.97935604],
       [ 0.99776725],
       [ 1.        ]])

In [185]:
training_set_scaled.shape

(4175, 1)

In [186]:
X_train = []
y_train = []
for i in range(30, len(training_set_scaled)):
    X_train.append(training_set_scaled[i-30:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)
X_train

array([[-0.83422711, -0.83399497, -0.81819218, ..., -0.74500727,
        -0.73564129, -0.71828525],
       [-0.83399497, -0.81819218, -0.81634767, ..., -0.73564129,
        -0.71828525, -0.71376896],
       [-0.81819218, -0.81634767, -0.80834076, ..., -0.71828525,
        -0.71376896, -0.71577386],
       ...,
       [ 0.83813976,  0.82434195,  0.85535661, ...,  0.96386133,
         0.97287279,  0.97756219],
       [ 0.82434195,  0.85535661,  0.86167944, ...,  0.97287279,
         0.97756219,  0.97935604],
       [ 0.85535661,  0.86167944,  0.86974121, ...,  0.97756219,
         0.97935604,  0.99776725]])

In [187]:
X_train.shape

(4145, 30)

In [188]:
y_train.shape

(4145,)

In [189]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
X_train.shape

(4145, 30, 1)

In [190]:
y_train = np.reshape(y_train, (y_train.shape[0], 1 ,1))
y_train.shape

(4145, 1, 1)

## Finding The Best Hyperparameters

### Best Optimizer

In [191]:
# def build_model(hp):
#     model = tf.keras.models.Sequential()
#     model.add(LSTM(units=120, activation='relu', return_sequences=True))
#     model.add(Dense(units=1, activation = 'linear'))
#     model.compile(optimizer = hp.Choice('optimizer', values = ['ftrl', 'adam', 'nadam', 'adamax', 'lion']),
#                   loss = 'mean_squared_error',
#                   metrics = ['accuracy']
#                   )
#     return model

In [192]:
# tuner = kt.RandomSearch(build_model,
#                         objective='accuracy',
#                         max_trials=6,
#                         directory='main',
#                         project_name='opt')

In [193]:
# tuner.search(X_train, y_train, epochs = 6)

In [194]:
# tuner.get_best_hyperparameters()[0].values

### Best Activation Function

In [195]:
# def build_model(hp):
#     model = tf.keras.models.Sequential()
#     for i in range(hp.Int('num_layers', min_value = 1, max_value = 12)):
#         model.add(Dense(units=87,
#                 activation = hp.Choice('activation_', values = ['relu', 'tanh', 'selu', 'leaky_relu', 'linear', 'softmax']))
#         )
#     model.add(Dense(units=1, activation = 'linear'))
#     model.compile(optimizer = 'lion',
#                   loss = 'mean_squared_error',
#                   metrics = ['root_mean_squared_error']
#                   )
#     return model

In [196]:
# tuner = kt.RandomSearch(build_model,
#                         objective='root_mean_squared_error',
#                         max_trials=6,
#                         directory='main',
#                         project_name='units')

In [197]:
# tuner.search(X_train, y_train, epochs = 6)

In [198]:
# tuner.get_best_hyperparameters()[0].values

### Optimal Number Of Units

In [199]:
# def build_model(hp):
#     model = tf.keras.models.Sequential()
#     model.add(LSTM(units=hp.Choice('Neurons', [60, 600, 12]), activation='relu', return_sequences=True))
#     model.add(LSTM(units=hp.Choice('Neurons', [60, 600, 12]), activation='relu', return_sequences=True))
#     model.add(LSTM(units=hp.Choice('Neurons', [60, 600, 12]), activation='relu', return_sequences=True))
#     model.add(LSTM(units=hp.Choice('Neurons', [60, 600, 12]), activation='relu'))
#     model.add(Dense(units=1, activation = 'linear'))
#     model.compile(optimizer = 'lion',
#                   loss = 'mean_squared_error',
#                   metrics = ['accuracy']
#                   )
#     return model

In [200]:
# tuner = kt.RandomSearch(build_model,
#                         objective='accuracy',
#                         max_trials=6,
#                         directory='main',
#                         project_name='neurons')

In [201]:
# tuner.search(X_train, y_train, epochs = 6)

In [202]:
# tuner.get_best_hyperparameters()[0].values

## Building The RNN with LSTM

In [203]:
rnn = Sequential()

In [204]:
rnn.add(LSTM(units=60, activation='relu', return_sequences=True, input_shape=(None, 1)))

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [205]:
rnn.add(LSTM(units=60, activation='relu', return_sequences=True))

In [206]:
rnn.add(LSTM(units=60, activation='relu', return_sequences=True))

In [207]:
rnn.add(LSTM(units=60, activation='relu', return_sequences=True))

In [208]:
rnn.add(LSTM(units=60, activation='relu'))

In [209]:
rnn.add(Dense(1, activation='linear'))

In [210]:
rnn.compile(optimizer='lion', loss='mean_squared_error')

In [211]:
rnn.fit(X_train, y_train, epochs = 9, batch_size = 24)

Epoch 1/9


173/173 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 0.0867
Epoch 2/9
173/173 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - loss: 0.0022
Epoch 3/9
173/173 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - loss: 0.0013
Epoch 4/9
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 8.7601e-04
Epoch 5/9
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 6.1491e-04
Epoch 6/9
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 4.7712e-04
Epoch 7/9
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 4.1995e-04
Epoch 8/9
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 3.7815e-04
Epoch 9/9
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 3.6044e-04


## Making a Test Set and Predicting the Outputs

In [212]:
inputs = dataset_train.iloc[-33:,1:2].values
inputs = sc.transform(inputs)
inputs.shape

(33, 1)

In [213]:
X_test = []
for i in range(30, 34):
    X_test.append(inputs[i-30:i, 0])
X_test = np.array(X_test)
X_test.shape

(4, 30)

In [214]:
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_stock_price = rnn.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step


In [215]:
predicted_stock_price

array([[24119.379],
       [24019.564],
       [23639.797],
       [23483.643]], dtype=float32)

In [216]:
from sklearn.metrics import mean_absolute_percentage_error
(1 - mean_absolute_percentage_error(dataset_train.iloc[-3:, 1:2].values, predicted_stock_price[:-1]))*100

99.57676054342525